In [32]:
import pandas as pd
import numpy as np
import collections
from numpy import linalg as LA

train = pd.read_csv("C:\Document\COURSEWORK\ML\hw1\propublicaTrain.csv")
test = pd.read_csv("C:\Document\COURSEWORK\ML\hw1\propublicaTest.csv")

nn = 3000
X_train = train.iloc[:nn, 1:-1]
y_train = train.iloc[:nn, 0]
X_test = train.iloc[nn:, 1:-1]
y_test = train.iloc[nn:, 0].values

# mean_list = []
# std_list = []
# for _, rows in X_train.iteritems():
#     mean_list.append(rows.mean())
#     std_list.append(rows.std())
# X_train = (X_train - mean_list)/std_list
# X_test = (X_test - mean_list)/std_list


In [33]:
class simple_knn():
    
    def __init__(self):
        pass
    
    def train(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X_test, k):
        distance = self.__dists__(X_test)
        num = distance.shape[0]
        y_pred = [0]*num
        
        k_y = self.__k_closest_y__(distance, k)
        
        for i in range(num):
            c = collections.Counter(k_y[i])
            y_pred[i] = c.most_common(1)[0][0]
        return np.array(y_pred)
    
    def __k_closest_y__(self, distance, k):
        k_y = []
        for dis in distance:
#             print(np.argsort(dis)[:k], len(k_y))
            k_y.append(self.y_train.loc[y_train.index.isin(np.argsort(dis)[:k])])
        return k_y
    
    def __dists__(self, X_test):
        dif = []
        for _, one_pass in X_test.iterrows():
            d = self.X_train - one_pass
            d = np.power((abs(d**1)).sum(axis=1), 1/1)
#             d = np.inner(self.X_train, one_pass)/(LA.norm(one_pass) * LA.norm(self.X_train))
#             d = []
#             for _, one_train in self.X_train.iterrows():
#                 d.append(np.inner(one_pass, one_train)/(LA.norm(one_train) * LA.norm(one_pass)))
            dif.append(d)
        return np.array(dif)

In [36]:
nn = simple_knn()
nn.train(X_train,y_train)
pred_y = nn.predict(X_test, k=10)
pred_y1 = nn.predict(X_train, k=10)
test_accu = np.sum((pred_y == y_test))/len(pred_y)
train_accu = np.sum((pred_y1 == y_train.values))/len(pred_y1)
print(train_accu, test_accu)

0.71 0.6512425021422451


In [27]:
class simple_mle():
    
    def __init__(self):
        pass
    
    def train(self, X, y):
        self.X_train = X
        self.y_train = y
        

    def predict(self, X_test):
        pred_y = np.zeros(len(X_test))
        
        class_one = self.X_train[self.y_train==1]
        class_zero = self.X_train[self.y_train==0]
        
        
        
        
        mu_one = class_one.mean()
        unbiased_one = class_one - mu_one
        sigma_one = self.estimate_sigma(unbiased_one)
        p_one = self.multi_normal_density(X_test, mu_one, sigma_one)
        
        mu_zero = class_zero.mean()
        unbiased_zero = class_zero - mu_zero
        sigma_zero = self.estimate_sigma(unbiased_zero)
        p_zero = self.multi_normal_density(X_test, mu_zero, sigma_zero)
        pred_y[p_one >= p_zero] = 1
        pred_y[p_one < p_zero] = 0
        return pred_y
    
    def estimate_sigma(self, X_training):
#         total = np.zeros((X_training.shape[1], X_training.shape[1]))
        total = 0
#         for _, rows in X_training.iterrows():
#             total += np.outer(rows, rows)
        for _, rows in X_training.iterrows():
            total += LA.norm(rows)
        print(total/len(X_training))
        return total/len(X_training)
    
    def multi_normal_density(self, X_testing, mu, sigma):
        X_testing -= mu
        sigma_norm = LA.norm(sigma)
        constant = 1/(np.power(2*np.pi, len(mu)/2) * sigma_norm**0.5)
#         U, s, Vh = LA.svd(sigma)
#         t = 1.0e-5
#         for ss in s:
#             if ss>t: ss = 1/ss
#             else: ss = 0
#         sigma_inv = np.dot(np.dot(Vh.T, np.diag(s)), U.T)
        X_norm = []
        for _, rows in X_testing.iterrows():
            X_norm.append(LA.norm(rows))
        ex = np.exp(-0.5*np.array(X_norm)/sigma_norm)
#         print(constant)
        return constant * ex

In [28]:
nn = simple_mle()
nn.train(X_train,y_train)
pred_y = nn.predict(X_test)
pred_y1 = nn.predict(X_train)
test_accu = np.sum((pred_y == y_test))/len(pred_y)
train_accu = np.sum((pred_y1 == y_train.values))/len(pred_y1)
print(train_accu, test_accu)

2.7691580400569245
2.5095445390300917
2.7691580400569245
2.5095445390300917
0.6543333333333333 0.6503856041131105


In [29]:
class simple_nb():
    
    def __init__(self):
        pass
    
    def train(self, X, y):
        self.X_train = X
        self.y_train = y
        

    def predict(self, X_test, alpha=1):
        pred_y = np.zeros(len(X_test))
        
        class_one = self.X_train[self.y_train==1]
        class_zero = self.X_train[self.y_train==0]
        
        p_y1 = len(class_one)/len(X_train)
        p_y0 = 1 - p_y1
        
        p_one, p_zero = [], []
        counter_one, counter_zero = [], []
        
        for _, col in class_one.iteritems():
            counter_one.append(self.count(col,class_one.shape[1], alpha))
        for _, col in class_zero.iteritems():
            counter_zero.append(self.count(col,class_zero.shape[1], alpha))
        
        for _, rows in X_test.iterrows():
            p_1, p_0 = 1, 1
            for i in range(len(rows)):
                if rows[i] in counter_one[i].keys():
                    p_1 *= counter_one[i][rows[i]]
                else:
                    p_1 /= (len(class_one) + class_one.shape[1]*alpha)
                if rows[i] in counter_zero[i].keys():
                    p_0 *= counter_zero[i][rows[i]]
                else:
                    p_0 /= (len(class_zero) + class_zero.shape[1]*alpha)
            p_one.append(p_1*p_y1)
            p_zero.append(p_0*p_y0)
    
        
        p_one = np.array(p_one)
        p_zero = np.array(p_zero)
        pred_y[p_one >= p_zero] = 1
        pred_y[p_one < p_zero] = 0
        return pred_y
    
    def count(self, col, num_rows, alpha):
        z = dict(collections.Counter(col))
        for i in z.keys():
            z[i] += alpha
            z[i] /= (len(col) + num_rows*alpha)
        return z

In [31]:
nn = simple_nb()
nn.train(X_train,y_train)
pred_y = nn.predict(X_test, alpha = 1)
pred_y1 = nn.predict(X_train, alpha = 1)
test_accu = np.sum((pred_y == y_test))/len(pred_y)
train_accu = np.sum((pred_y1 == y_train.values))/len(pred_y1)
print(train_accu, test_accu)

0.6686666666666666 0.6546700942587832


In [30]:
# np.inner(X_test.iloc[3,:], X_test.iloc[3,:])
# LA.norm(X_test.iloc[3,:])**2
# hh = X_train.iloc[:10, :]
# z = np.outer(hh, hh)
# total = np.zeros((hh.values.shape[1],hh.values.shape[1]))
# for _ , rows in hh.iterrows():
#     total += np.outer(rows, rows)
# print(z.shape, total)
# # hh.values.shape
# a = np.array([[1,6,7],[2,12,14],[3,18,21]])
# i = [[2, 1, 3],[4,2,1],[5,0,2]]
# np.dot(np.dot(a, i), a.T)
# # b = np.array([1,6,7])
# # np.dot(np.dot(b, i), b.T) 
# np.diag(np.diag(a))

z = dict(collections.Counter(X_train))
for i in z.keys():
    print(i)

sex
age
race
juv_fel_count
juv_misd_count
juv_other_count
priors_count
c_charge_degree_F
c_charge_degree_M
